In [1]:
#2. Deep Hash
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image


# Load the pretrained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def hash_image_sha3_512(image_path):

    # Load the image
    image_path = "Capture.JPG"  # Replace with your image path
    img = image.load_img(image_path, target_size=(224, 224))  # Resize image to 224x224
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract features
    features = base_model.predict(img_array)

    # Convert features to a binary hash
    binary_threshold = 0.0  # Set a threshold to binarize the features
    hash_code = (features > binary_threshold).astype(int)

    print("Deep Hash Code:", hash_code)

    return hash_code

C:\Users\juanc\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\juanc\AppData\Local\Temp\ipykernel_5864\3555196728.py:10: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
# 3. Sign the hash of an image

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.backends import default_backend

def sign_image(image_path, private_key):
    # Get the current process
    process = psutil.Process()

    # Record CPU times before function execution
    cpu_times_before = process.cpu_times()

    start_time = time.time()
    gpus_before = GPUtil.getGPUs()
    memory_used_before = sum(gpu.memoryUsed for gpu in gpus_before)
    gpu_load_before = sum(gpu.load for gpu in gpus_before)

    image_hash = hash_image_sha3_512(image_path)

    # Sign the hash with the private key
    signature = private_key.sign(
        image_hash,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA512()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA512()
    )
    gpus_after = GPUtil.getGPUs()

    # Record CPU times after function execution
    end_time = time.time()

    cpu_times_after = process.cpu_times()
    memory_used_after = sum(gpu.memoryUsed for gpu in gpus_after)
    gpu_load_after = sum(gpu.load for gpu in gpus_after)

    memory_diff = memory_used_after - memory_used_before
    gpu_load_diff = gpu_load_after - gpu_load_before
    user_time = cpu_times_after.user - cpu_times_before.user
    system_time = cpu_times_after.system - cpu_times_before.system
    total_time = end_time - start_time

    # Print results
    print(f"User CPU time: {user_time:.4f} seconds")
    print(f"System CPU time: {system_time:.4f} seconds")
    print(f"Total elapsed time: {total_time:.4f} seconds")
    print(f"CPU usage: {(user_time + system_time) / total_time * 100:.2f}%")
    print(f"GPU memory used: {memory_diff:.2f} MB")
    print(f"GPU load change: {gpu_load_diff:.2f}")
    GPUtil.showUtilization(all=False, attrList=None, useOldCode=False)
    return signature

In [3]:
# 4. Verify the signature
def verify_image_signature(image_path, signature, public_key):


    # Get the current process
    process = psutil.Process()

    # Record CPU times before function execution
    cpu_times_before = process.cpu_times()

    start_time = time.time()
    gpus_before = GPUtil.getGPUs()
    memory_used_before = sum(gpu.memoryUsed for gpu in gpus_before)
    gpu_load_before = sum(gpu.load for gpu in gpus_before)

    with open(image_path, "rb") as img_file:
        image_data = img_file.read()

    image_hash = hash_image_sha3_512(image_path)

    # Verify the signature with the public key
    try:
        public_key.verify(
            signature,
            image_hash,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("Signature is valid!")
    except Exception as e:
        print(f"Signature verification failed: {e}")


    # Record CPU times after function execution
    end_time = time.time()

    cpu_times_after = process.cpu_times()
    gpus_after = GPUtil.getGPUs()
    memory_used_after = sum(gpu.memoryUsed for gpu in gpus_after)
    gpu_load_after = sum(gpu.load for gpu in gpus_after)

    memory_diff = memory_used_after - memory_used_before
    gpu_load_diff = gpu_load_after - gpu_load_before
    user_time = cpu_times_after.user - cpu_times_before.user
    system_time = cpu_times_after.system - cpu_times_before.system
    total_time = end_time - start_time

    # Print results
    print(f"User CPU time: {user_time:.4f} seconds")
    print(f"System CPU time: {system_time:.4f} seconds")
    print(f"Total elapsed time: {total_time:.4f} seconds")
    print(f"CPU usage: {(user_time + system_time) / total_time * 100:.2f}%")
    print(f"GPU memory used: {memory_diff:.2f} MB")
    print(f"GPU load change: {gpu_load_diff:.2f}")
    GPUtil.showUtilization(all=False, attrList=None, useOldCode=False)

# Load signature and verify
#with open("image_signature.sig", "rb") as sig_file:
#    saved_signature = sig_file.read()

#verify_image_signature("Capture.JPG", saved_signature, public_key)

In [4]:
!pip install gputil

In [5]:
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.backends import default_backend
import time
import psutil
import GPUtil

def main():
    # Example for image hash
    image_path = "Capture.JPG"

    # Load the private key from the file
    with open("private_key.pem", "rb") as key_file:
        private_key = load_pem_private_key(
            key_file.read(),
            password=None,  # Add a password here if the private key is encrypted
            backend=default_backend()
        )

    with open("public_key.pem", "rb") as key_file:
        public_key = load_pem_public_key(
          key_file.read(),
          backend=default_backend()
        )

    start_time = time.time()
    #hashed = hash_image_sha3_512("Capture.JPG")

    signature = sign_image(image_path, private_key)

    # Save signature to a file
    with open("image_signature.sig", "wb") as sig_file:
        sig_file.write(signature)

    print("Image signed successfully. Signature saved to 'image_signature.sig'.")

    verify = verify_image_signature(image_path, signature, public_key)

    #print("Hash value:", hash_value)
    print("--- %s seconds ---" % (time.time() - start_time))

In [10]:
main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Deep Hash Code: [[0 0 1 ... 1 1 1]]
User CPU time: 0.0312 seconds
System CPU time: 0.0156 seconds
Total elapsed time: 0.1697 seconds
CPU usage: 27.61%
GPU memory used: 0.00 MB
GPU load change: 0.03
| ID | GPU | MEM |
------------------
|  0 | 34% | 12% |
Image signed successfully. Signature saved to 'image_signature.sig'.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Deep Hash Code: [[0 0 1 ... 1 1 1]]
Signature verification failed: 
User CPU time: 0.0469 seconds
System CPU time: 0.0000 seconds
Total elapsed time: 0.1169 seconds
CPU usage: 40.10%
GPU memory used: 0.00 MB
GPU load change: 0.01
| ID | GPU | MEM |
------------------
|  0 | 35% | 12% |
--- 0.41774630546569824 seconds ---
